Import packages:

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
from see import Segmentors
from see import JupyterGUI

Download data from KOMATSUNA:

In [ ]:
#The following code will download the dataset to the Images_Data folder
from see import DataDownload as dd

dd.downloadKOMATSUNA(folder='./Image_Data/KOMATSUNA/')

In [ ]:
from imageio import v3 as imageio

img1 = imageio.imread('Image_data/KOMATSUNA/rgbd_plant/rgb_00_000_00.png')
gmask1 = imageio.imread('Image_data/KOMATSUNA/rgbd_label/label_00_000_00.png')

img2 = imageio.imread('Image_data/KOMATSUNA/rgbd_plant/rgb_00_000_01.png')
gmask2 = imageio.imread('Image_data/KOMATSUNA/rgbd_label/label_00_000_01.png')

img3 = imageio.imread('Image_data/KOMATSUNA/rgbd_plant/rgb_00_000_02.png')
gmask3 = imageio.imread('Image_data/KOMATSUNA/rgbd_label/label_00_000_02.png')

img4 = imageio.imread('Image_data/KOMATSUNA/rgbd_plant/rgb_00_000_03.png')
gmask4 = imageio.imread('Image_data/KOMATSUNA/rgbd_label/label_00_000_03.png')


In [ ]:
from see import base_classes

# put data in a pipeline
mydata = base_classes.pipedata()
mydata.append(img1)
mydata.append(img2)
mydata.append(img3)
mydata.append(img4)
mydata.gtruth = []
mydata.gtruth.append(gmask1)
mydata.gtruth.append(gmask2)
mydata.gtruth.append(gmask3)
mydata.gtruth.append(gmask4)

----
<a name="Genetic_Algorithm_Search"></a>

## Genetic Algorithm Search

First import image files, as well as the following packages:

In [ ]:
from see.Segmentors import segmentor
from see.ColorSpace import colorspace
from see.Workflow import workflow
from see.Segment_Fitness import segment_fitness
from see import base_classes, GeneticSearch

#define an algorithm workflow
workflow.addalgos([colorspace, segmentor, segment_fitness])

To run the genetic algorithm, we need to initialize an instance of an evolver. The original image and ground truth segmentation image are inputs to it, along with an integer value for population size. This value sets how many indivudals are in our population. For this example, we'll set this number to be equal to 10.

In [ ]:
#TODO This needs to be fixed Using the new syntax.
#my_evolver = GeneticSearch.Evolver(workflow, mydata, pop_size=10)
my_evolver = GeneticSearch.Evolver(workflow, mydata, pop_size=10)

Now that the evolver has been initialized, we can run the genetic algorithm for a specified number of generations (or iterations). Here we will set this number equal to 5.

In [ ]:
population = my_evolver.run(ngen=5)

In [ ]:
print(my_evolver.hof[0].fitness)

----
<a name="Reading_the_Results"></a>

## Reading the Results

After the genetic algorithm is complete, we can retrieve the individuals that resulted in the lowest (best) fitness values by printing `my_evolver.hof`. These individuals are sorted according to fitness value, so to get the overal best individual, we can simply look at the first individual in the list. 

In [ ]:
params = my_evolver.hof[0]
fitness = params.fitness.values[0]
print(f'Best Individual ({fitness}):\n', params)

We can see the mask this individual generates by evaluating it, then plotting the result:

In [ ]:
seg = workflow(params)
data = seg.runAlgo(mydata)

In [ ]:
for i in range(len(data)-1):
    plt.figure(figsize=(20, 10))
    plt.subplot(131)
    plt.imshow(data[i])
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(132)
    plt.imshow(data.mask[i])
    plt.title("Segmentation")
    plt.axis('off')

    plt.subplot(133)
    plt.imshow(data.gtruth[i])
    plt.title("Ground Truth")
    plt.axis('off')


plt.tight_layout
plt.show()

In [ ]:
len(data)